In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets

We will build a simple Multi Layer Perceptron to classify images in Fashion-MNIST dataset, which contains images of 10 different categories.

In [2]:
#Let's download data using torchvision
trainset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = True, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

testset = datasets.FashionMNIST('./../0. Data/', 
                                 download = True, 
                                 train = False, 
                                 transform = T.Compose([
                                     T.ToTensor()
                                 ]))

#split training data to training and validation  data
train_set, val_set = torch.utils.data.random_split(trainset, [50000, 10000])

#Convert data to dataloader
train_loader = torch.utils.data.DataLoader(train_set, 
                                          batch_size = 32, 
                                          shuffle = True)

val_loader = torch.utils.data.DataLoader(val_set,
                                        batch_size = 32,
                                        shuffle = True)

test_loader = torch.utils.data.DataLoader(testset, 
                                         batch_size = 32, 
                                         shuffle = True)

full_train_set  = torch.utils.data.DataLoader(trainset, 
                                          batch_size = 32, 
                                          shuffle = True)


In [3]:
#Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
#Create a MLP class defining our neural network
class MLP(nn.Module):
    def __init__(self, input_len, output_len):
        super(MLP, self).__init__()
        #three fully connected layers
        self.fc1 = nn.Linear(in_features=input_len, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)
        
        
    def forward(self, x):
        #convert image to a one dimentional tensor before feeding to neural network
        x = x.flatten(start_dim=1)
        #activation function is relu
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        

In [5]:
#define training function
def train(Model, validate, max_epoch):
    for epoch in range(max_epoch):
        Train_Loss = []
        Val_Loss =[]
        loader = full_train_set
        
        if(validate):
            loader = train_loader
        
        cnf_tr = torch.zeros(10,10)
        cnf_val = torch.zeros(10,10)
        
        #Train on training data
        for i, sample in enumerate(loader):

            #set model to train mode
            Model.train()
            #set gradiuents to zero
            optimizer.zero_grad()
            #obtain output
            output = Model(sample[0].to(device)).to(device)
            #compute loss
            loss = loss_function(output, sample[1].to(device))
            #compute gradients
            loss.backward()
            #optimize weights
            optimizer.step()
            #record train loss
            Train_Loss.append(loss.item())
            
            with torch.no_grad():
                #calculate output by argmax
                output = torch.argmax(output, 1)
                #update entries in confusion matrix
                for i in range(output.shape[0]):
                    cnf_tr[output[i],sample[1][i]] +=1
            
        if(validate):
            #Evaluate on validation data
            with torch.no_grad():
                #set model to evaluation mode
                Model.eval()
                #evaluate on tvaidation data
                for i, sample in enumerate(val_loader):
                    output = Model(sample[0].to(device))
                    loss = loss_function(output, sample[1].to(device))
                    Val_Loss.append(loss.item())
                    #calculate output by argmax
                    output = torch.argmax(output, 1)
                    #update entries in confusion matrix
                    for i in range(output.shape[0]):
                        cnf_val[output[i],sample[1][i]] +=1
                   
        actual_count = torch.sum(cnf_tr, dim=0)
        correct_pred = torch.tensor([cnf_tr[i,i] for i in range(10)])
        A_tr = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        if(validate):
            actual_count = torch.sum(cnf_val, dim=0)
            correct_pred = torch.tensor([cnf_val[i,i] for i in range(10)])
            A_val = (torch.sum(correct_pred)/torch.sum(actual_count)).item()
        
        #print losses in every epoch
        if(validate):
            print('epoch : ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Val_acc : ', np.round(A_val,4),  
                  '; Train_loss  ',np.round(np.mean(Train_Loss),4),  '; Val_loss  ',np.round(np.mean(Val_Loss),4))
        else:
            print('epoch = ',epoch,'; Train_acc : ', np.round(A_tr,4), '; Train_loss  ',np.round(np.mean(Train_Loss),4))

In [6]:
#Function top evaluate model using performace metrices
def evaluate(cnf):
    actual_count = torch.sum(cnf, dim=0)
    predicted_count = torch.sum(cnf, dim=1)
    correct_pred = torch.tensor([cnf[i,i] for i in range(10)])
    #Precision
    precision = correct_pred/predicted_count
    #Recall
    recall = correct_pred/actual_count
    #F1-Score
    f1_score = 2*precision*recall/(precision+recall)
    #Accuracy
    Accuracy = torch.sum(correct_pred)/torch.sum(actual_count)
    print('\n',pd.DataFrame({'Class':[i for i in range(10)],
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1_Score': f1_score}))
    
    
    print('\nAccuracy  : ', Accuracy.item())
    

In [7]:
#function to test model
def test(Model):
    Loss = []
    #confusion matrix
    cnf = torch.zeros(10,10)

    #evaluate on test data
    with torch.no_grad():
        #set model to evaluation mode
        Model.eval()
        #evaluate on test data
        for i, sample in enumerate(test_loader):
            output = Model(sample[0].to(device))
            loss = loss_function(output, sample[1].to(device))
            Loss.append(loss.item())
            #calculate output by argmax
            output = torch.argmax(output, 1)
            #update entries in confusion matrix
            for i in range(output.shape[0]):
                cnf[output[i],sample[1][i]] +=1

        #print test loss
        print('Test loss : ', np.mean(Loss))

    #print evaluation summary
    evaluate(cnf)

In [8]:
#define loss function
loss_function = nn.CrossEntropyLoss()

In [9]:
#Create Model
Model = MLP(784,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train model with validation
train(Model, validate=True, max_epoch=30)

epoch :  0 ; Train_acc :  0.8168 ; Val_acc :  0.8486 ; Train_loss   0.5014 ; Val_loss   0.3994
epoch :  1 ; Train_acc :  0.8632 ; Val_acc :  0.861 ; Train_loss   0.3697 ; Val_loss   0.3749
epoch :  2 ; Train_acc :  0.8772 ; Val_acc :  0.8734 ; Train_loss   0.3329 ; Val_loss   0.3355
epoch :  3 ; Train_acc :  0.8862 ; Val_acc :  0.8841 ; Train_loss   0.3056 ; Val_loss   0.3145
epoch :  4 ; Train_acc :  0.8921 ; Val_acc :  0.8832 ; Train_loss   0.2858 ; Val_loss   0.3136
epoch :  5 ; Train_acc :  0.8991 ; Val_acc :  0.8827 ; Train_loss   0.2682 ; Val_loss   0.3296
epoch :  6 ; Train_acc :  0.902 ; Val_acc :  0.8907 ; Train_loss   0.2581 ; Val_loss   0.2958
epoch :  7 ; Train_acc :  0.9086 ; Val_acc :  0.883 ; Train_loss   0.243 ; Val_loss   0.3121
epoch :  8 ; Train_acc :  0.9118 ; Val_acc :  0.8835 ; Train_loss   0.2322 ; Val_loss   0.3271
epoch :  9 ; Train_acc :  0.9142 ; Val_acc :  0.8865 ; Train_loss   0.2224 ; Val_loss   0.3269
epoch :  10 ; Train_acc :  0.9178 ; Val_acc :  0.8929 

In [10]:
#Model's validation loss starts increasing after 12 epochs,
#So, Let's train our model for 12 epochs on full training set
#Create Model
Model = MLP(784,10).to(device)
#Define optimizer
optimizer = optim.Adam(Model.parameters())
#train with no validation
train(Model, validate=False, max_epoch=12)

epoch =  0 ; Train_acc :  0.8263 ; Train_loss   0.4824
epoch =  1 ; Train_acc :  0.8689 ; Train_loss   0.3572
epoch =  2 ; Train_acc :  0.8792 ; Train_loss   0.3226
epoch =  3 ; Train_acc :  0.8892 ; Train_loss   0.2958
epoch =  4 ; Train_acc :  0.896 ; Train_loss   0.2788
epoch =  5 ; Train_acc :  0.9001 ; Train_loss   0.2655
epoch =  6 ; Train_acc :  0.9061 ; Train_loss   0.2505
epoch =  7 ; Train_acc :  0.9094 ; Train_loss   0.2397
epoch =  8 ; Train_acc :  0.9136 ; Train_loss   0.2289
epoch =  9 ; Train_acc :  0.9169 ; Train_loss   0.2204
epoch =  10 ; Train_acc :  0.92 ; Train_loss   0.2108
epoch =  11 ; Train_acc :  0.9227 ; Train_loss   0.2028


In [11]:
#Let's test model now
test(Model)

Test loss :  0.33960034710149795

    Class  Precision  Recall  F1_Score
0      0   0.841642   0.861  0.851211
1      1   0.983000   0.983  0.983000
2      2   0.774931   0.847  0.809365
3      3   0.851005   0.931  0.889207
4      4   0.852535   0.740  0.792291
5      5   0.976721   0.965  0.970825
6      6   0.756906   0.685  0.719160
7      7   0.941748   0.970  0.955665
8      8   0.968411   0.981  0.974665
9      9   0.970588   0.957  0.963746

Accuracy  :  0.8920000195503235


In [12]:
#Finally, let's save our model
torch.save(Model.state_dict(), './saved_models/model_simple_MLP.pth')

In [13]:
#Let's retireve and make prediction once again
Modelx = MLP(784,10)
Modelx.load_state_dict(torch.load('./saved_models/model_simple_MLP.pth'))

<All keys matched successfully>

In [14]:
#send model to GPU
Modelx.to(device)
Loss = []
#confusion matrix
cnf = torch.zeros(10,10)

#evaluate on test data
with torch.no_grad():
    #set model to evaluation mode
    Modelx.eval()
    #evaluate on test data
    for i, sample in enumerate(test_loader):
        output = Modelx(sample[0].to(device))
        loss = loss_function(output, sample[1].to(device))
        Loss.append(loss.item())
        #calculate output by argmax
        output = torch.argmax(output, 1)
        #update entries in confusion matrix
        for i in range(output.shape[0]):
            cnf[output[i],sample[1][i]] +=1
            
    #print test loss
    print('Test loss : ', np.mean(Loss))

#print evaluation summary
evaluate(cnf)

Test loss :  0.3392818896843793

    Class  Precision  Recall  F1_Score
0      0   0.841642   0.861  0.851211
1      1   0.983000   0.983  0.983000
2      2   0.774931   0.847  0.809365
3      3   0.851005   0.931  0.889207
4      4   0.852535   0.740  0.792291
5      5   0.976721   0.965  0.970825
6      6   0.756906   0.685  0.719160
7      7   0.941748   0.970  0.955665
8      8   0.968411   0.981  0.974665
9      9   0.970588   0.957  0.963746

Accuracy  :  0.8920000195503235
